In [18]:
# Import libaries
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen

### Data read in

In [103]:
no_dupes = pd.read_csv('./data/imdb_titles_no_dupes.csv')

In [104]:
no_dupes.shape

(1886, 6)

In [105]:
no_dupes.head()

,name,href,verification,input_genre,scraped_genres,description
0,Avatar: The Last Airbender,/title/tt0417299/,1. Avatar: The Last Airbender (2005–2008),Action,"Animation, Action, Adventure","In a war-torn world of elemental magic, a youn..."
1,Game of Thrones,/title/tt0944947/,2. Game of Thrones (2011–2019),Action,"Action, Adventure, Drama",Nine noble families fight for control over the...
2,Fullmetal Alchemist: Brotherhood,/title/tt1355642/,3. Fullmetal Alchemist: Brotherhood (2009–2010),Action,"Animation, Action, Adventure",Two brothers search for a Philosopher's Stone ...
3,Attack on Titan,/title/tt2560140/,4. Attack on Titan (2013–2023),Action,"Animation, Action, Adventure",After his hometown is destroyed and his mother...
4,Arcane,/title/tt11126994/,5. Arcane (2021– ),Action,"Animation, Action, Adventure",Set in Utopian Piltover and the oppressed unde...


In [102]:
base_url = 'https://www.imdb.com'

test_df = no_dupes.head().copy()

### Convert back to list of dictionaries

In [95]:
# Technique reference: https://www.geeksforgeeks.org/create-a-list-from-rows-in-pandas-dataframe/

# Create an empty list
entries_list =[]

# Iterate over each row
for index, rows in test_df.iterrows():
    # Create list for the current row
    my_dict ={'name': rows['name'], 'href' : rows['href'], 'verifiation' : rows['verification'], 'scrape_genre' : rows['scrape_genre']}
    
    # append the list to the final list
    entries_list.append(my_dict)
  
# Print the list
# print(entries_list)

### Iterate thru list and scrape each URL for descriptions and genre tags

In [99]:
base_url = 'https://www.imdb.com'

for entry in entries_list:
    full_url = base_url + entry['href']
    # print(full_url)

    # Technique reference to avoid 403 error: 
    # https://medium.com/@raiyanquaium/how-to-web-scrape-using-beautiful-soup-in-python-without-running-into-http-error-403-554875e5abed

    req = Request(full_url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    soup = BeautifulSoup(webpage, "html.parser")
    
    description_obj = soup.find('p')
    description = description_obj.text

    genre_tags = []
    span_obj = soup.find_all('span', attrs='ipc-chip__text')

    for item in span_obj:
        if item.text != 'Back to top':
            genre_tags.append(item.text)
    
    entry['description'] = description
    entry['genre_tags'] = genre_tags


https://www.imdb.com/title/tt0417299/
https://www.imdb.com/title/tt0944947/
https://www.imdb.com/title/tt1355642/
https://www.imdb.com/title/tt2560140/
https://www.imdb.com/title/tt11126994/


In [100]:
entries_list

[{'name': 'Avatar: The Last Airbender',
  'href': '/title/tt0417299/',
  'verifiation': '1. Avatar: The Last Airbender (2005–2008)',
  'genre': 'Action',
  'description': 'In a war-torn world of elemental magic, a young boy reawakens to undertake a dangerous mystic quest to fulfill his destiny as the Avatar, and bring peace to the world.In a war-torn world of elemental magic, a young boy reawakens to undertake a dangerous mystic quest to fulfill his destiny as the Avatar, and bring peace to the world.In a war-torn world of elemental magic, a young boy reawakens to undertake a dangerous mystic quest to fulfill his destiny as the Avatar, and bring peace to the world.',
  'genre_tags': ['Animation', 'Action', 'Adventure']},
 {'name': 'Game of Thrones',
  'href': '/title/tt0944947/',
  'verifiation': '2. Game of Thrones (2011–2019)',
  'genre': 'Action',
  'description': 'Nine noble families fight for control over the lands of Westeros, while an ancient enemy returns after being dormant fo

In [101]:
pd.DataFrame(entries_list)

,name,href,verifiation,genre,description,genre_tags
0,Avatar: The Last Airbender,/title/tt0417299/,1. Avatar: The Last Airbender (2005–2008),Action,"In a war-torn world of elemental magic, a youn...","[Animation, Action, Adventure]"
1,Game of Thrones,/title/tt0944947/,2. Game of Thrones (2011–2019),Action,Nine noble families fight for control over the...,"[Action, Adventure, Drama]"
2,Fullmetal Alchemist: Brotherhood,/title/tt1355642/,3. Fullmetal Alchemist: Brotherhood (2009–2010),Action,Two brothers search for a Philosopher's Stone ...,"[Animation, Action, Adventure]"
3,Attack on Titan,/title/tt2560140/,4. Attack on Titan (2013–2023),Action,After his hometown is destroyed and his mother...,"[Animation, Action, Adventure]"
4,Arcane,/title/tt11126994/,5. Arcane (2021– ),Action,Set in Utopian Piltover and the oppressed unde...,"[Animation, Action, Adventure]"


In [46]:
# Alternate method

# # Technique reference: https://stackoverflow.com/questions/36768068/get-meta-tag-content-property-with-beautifulsoup-and-python

# title = soup.find("meta", property="og:title")
# url = soup.find("meta", property="og:url")
# description = soup.find("meta", property="og:description")

# print(title["content"] if title else "No meta title given")
# print(url["content"] if url else "No meta url given")
# print(description["content"] if description else "No meta description given")

Avatar: The Last Airbender (TV Series 2005–2008) - IMDb
https://www.imdb.com/title/tt0417299/
Avatar: The Last Airbender: Created by Michael Dante DiMartino, Bryan Konietzko. With Dee Bradley Baker, Zach Tyler Eisen, Mae Whitman, Jack De Sena. In a war-torn world of elemental magic, a young boy reawakens to undertake a dangerous mystic quest to fulfill his destiny as the Avatar, and bring peace to the world.
